In [1]:
import submitit

import torch
import random
import numpy as np

import pickle
import itertools
import argparse
import logging
import os
import pathlib
import time
import json
import math
import matplotlib.pyplot as plt
from torch.utils import data
from fvcore.common.config import CfgNode
from submitit.core.utils import FailedJobError
from itertools import cycle
from time import sleep
from multi_objective.main import main, get_config
from plotting.plotting import load_data, plot_row, generate_table, generate_table_taskwise

In [2]:
executor = submitit.AutoExecutor(folder="tmp/submitit")
seeds = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [3]:
executor.update_parameters(timeout_min=120, slurm_partition="alldlc_gpu-rtx2080", name='results', gpus_per_node=1)
executor.update_parameters(slurm_array_parallelism=100)

In [4]:
def percent_finished(jobs):
    if len(jobs):
        return sum(job.done() for job in jobs) / len(jobs)

def execute(config, seeds):
    cfg = config.clone()
    cfg.eval_every = 100
    cfg.test_eval_every = 100  # generate test results
    cfg.metrics = ['mcr', 'mcr']
    
    cfgs = []
    for seed in seeds:
        cfg = cfg.clone()
        cfg.merge_from_list(['seed', seed])
        cfgs.append(cfg)
    
    tags = [f"result_{s :02d}" for s in seeds]
    
    # func, rank, world_size, cfg, tag
    return executor.map_array(main, cycle([0]), cycle([1]), cfgs, tags)

# Baselines

#### cosmos

In [6]:
jobs = execute(get_config('configs/multi_mnist/cosmos.yaml'), seeds)

In [35]:
jobs = execute(get_config('configs/multi_fashion/cosmos.yaml'), seeds)

In [36]:
jobs = execute(get_config('configs/multi_fashion_mnist/cosmos.yaml'), seeds)

In [7]:
jobs = execute(get_config('configs/multi_mnist/cosmos_orig.yaml'), seeds)

In [38]:
jobs = execute(get_config('configs/multi_fashion/cosmos_orig.yaml'), seeds)

In [39]:
jobs = execute(get_config('configs/multi_fashion_mnist/cosmos_orig.yaml'), seeds)

#### mgda

In [40]:
jobs = execute(get_config('configs/multi_mnist/mgda.yaml'), seeds)

In [41]:
jobs = execute(get_config('configs/multi_fashion/mgda.yaml'), seeds)

In [42]:
jobs = execute(get_config('configs/multi_fashion_mnist/mgda.yaml'), seeds)

#### phn

In [43]:
jobs = execute(get_config('configs/multi_mnist/phn.yaml'), seeds)

In [44]:
jobs = execute(get_config('configs/multi_fashion/phn.yaml'), seeds)

In [45]:
jobs = execute(get_config('configs/multi_fashion_mnist/phn.yaml'), seeds)

In [46]:
jobs = execute(get_config('configs/multi_mnist/phn_orig.yaml'), seeds)

In [47]:
jobs = execute(get_config('configs/multi_fashion/phn_orig.yaml'), seeds)

In [48]:
jobs = execute(get_config('configs/multi_fashion_mnist/phn_orig.yaml'), seeds)

#### pmtl

In [49]:
jobs = execute(get_config('configs/multi_mnist/pmtl.yaml'), seeds)

In [50]:
jobs = execute(get_config('configs/multi_fashion/pmtl.yaml'), seeds)

In [51]:
jobs = execute(get_config('configs/multi_fashion_mnist/pmtl.yaml'), seeds)

#### single task

In [52]:
jobs = execute(get_config('configs/multi_mnist/single_task_1.yaml'), seeds)
jobs = execute(get_config('configs/multi_mnist/single_task_2.yaml'), seeds)

In [53]:
jobs = execute(get_config('configs/multi_fashion/single_task_1.yaml'), seeds)
jobs = execute(get_config('configs/multi_fashion/single_task_2.yaml'), seeds)

In [54]:
jobs = execute(get_config('configs/multi_fashion_mnist/single_task_1.yaml'), seeds)
jobs = execute(get_config('configs/multi_fashion_mnist/single_task_2.yaml'), seeds)

#### uniform

In [55]:
jobs = execute(get_config('configs/multi_mnist/uniform.yaml'), seeds)

In [56]:
jobs = execute(get_config('configs/multi_fashion/uniform.yaml'), seeds)

In [57]:
jobs = execute(get_config('configs/multi_fashion_mnist/uniform.yaml'), seeds)

## Results
#### Loss

In [5]:
results = load_data(dirname='results')
plot_row(results, prefix='baselines')
generate_table(results, name='tab-baselines_loss')
generate_table_taskwise(results, name='tab-baselines_loss_tw')

loaded data for multi_mnist
loaded data for multi_fashion
loaded data for multi_fashion_mnist
success. See baselines.pdf


#### MCR

In [6]:
results = load_data(dirname='results', custom_metric=True)
plot_row(results, prefix='baselines_mcr')
generate_table(results, name='tab-baselines_mcr')
generate_table_taskwise(results, name='tab-baselines_mcr_tw')

loaded data for multi_mnist
loaded data for multi_fashion
loaded data for multi_fashion_mnist
success. See baselines_mcr.pdf


# Different sizes

In [25]:
executor.update_parameters(timeout_min=400)

#### size 50

In [18]:
jobs = execute(get_config('size_50_configs/multi_fashion/uniform.yaml'), seeds)

In [19]:
jobs = execute(get_config('size_50_configs/multi_fashion/single_task_1.yaml'), seeds)
jobs = execute(get_config('size_50_configs/multi_fashion/single_task_2.yaml'), seeds)

In [20]:
jobs = execute(get_config('size_50_configs/multi_fashion_mnist/uniform.yaml'), seeds)

In [21]:
jobs = execute(get_config('size_50_configs/multi_fashion_mnist/single_task_1.yaml'), seeds)
jobs = execute(get_config('size_50_configs/multi_fashion_mnist/single_task_2.yaml'), seeds)

In [26]:
jobs = execute(get_config('size_50_configs/multi_mnist/uniform.yaml'), seeds)

In [27]:
jobs = execute(get_config('size_50_configs/multi_mnist/single_task_1.yaml'), seeds)
jobs = execute(get_config('size_50_configs/multi_mnist/single_task_2.yaml'), seeds)

#### size 10

In [22]:
jobs = execute(get_config('size_10_configs/multi_fashion/uniform.yaml'), seeds)

In [23]:
jobs = execute(get_config('size_10_configs/multi_fashion/single_task_1.yaml'), seeds)
jobs = execute(get_config('size_10_configs/multi_fashion/single_task_2.yaml'), seeds)

In [10]:
jobs = execute(get_config('size_10_configs/multi_fashion_mnist/uniform.yaml'), seeds)

In [25]:
jobs = execute(get_config('size_10_configs/multi_fashion_mnist/single_task_1.yaml'), seeds)
jobs = execute(get_config('size_10_configs/multi_fashion_mnist/single_task_2.yaml'), seeds)

In [26]:
jobs = execute(get_config('size_10_configs/multi_mnist/uniform.yaml'), seeds)

In [27]:
jobs = execute(get_config('size_10_configs/multi_mnist/single_task_1.yaml'), seeds)
jobs = execute(get_config('size_10_configs/multi_mnist/single_task_2.yaml'), seeds)

#### size 0.5

In [28]:
jobs = execute(get_config('size_0.5_configs/multi_fashion/uniform.yaml'), seeds)

In [29]:
jobs = execute(get_config('size_0.5_configs/multi_fashion/single_task_1.yaml'), seeds)
jobs = execute(get_config('size_0.5_configs/multi_fashion/single_task_2.yaml'), seeds)

In [30]:
jobs = execute(get_config('size_0.5_configs/multi_fashion_mnist/uniform.yaml'), seeds)

In [31]:
jobs = execute(get_config('size_0.5_configs/multi_fashion_mnist/single_task_1.yaml'), seeds)
jobs = execute(get_config('size_0.5_configs/multi_fashion_mnist/single_task_2.yaml'), seeds)

In [32]:
jobs = execute(get_config('size_0.5_configs/multi_mnist/uniform.yaml'), seeds)

In [33]:
jobs = execute(get_config('size_0.5_configs/multi_mnist/single_task_1.yaml'), seeds)
jobs = execute(get_config('size_0.5_configs/multi_mnist/single_task_2.yaml'), seeds)

In [9]:
jobs = execute(get_config('size_10_configs/multi_fashion_mnist/single_task_2.yaml'), seeds)

## Results
#### Loss 0.5

In [13]:
results = load_data(dirname='results_size_0.5', custom_metric=True)
plot_row(results, prefix='size_0.5')
generate_table(results, name='tab-size_0.5')

loaded data for multi_mnist
loaded data for multi_fashion
loaded data for multi_fashion_mnist
success. See size_0.5.pdf


#### Loss 10

In [14]:
results = load_data(dirname='results_size_10', custom_metric=True)
plot_row(results, prefix='size_10')
generate_table(results, name='tab-size_10')

loaded data for multi_mnist
loaded data for multi_fashion
loaded data for multi_fashion_mnist
success. See size_10.pdf


#### Loss 50

In [28]:
results = load_data(dirname='results_size_50', custom_metric=True)
plot_row(results, prefix='size_50')
generate_table(results, name='tab-size_50')

loaded data for multi_mnist
loaded data for multi_fashion
loaded data for multi_fashion_mnist
success. See size_50.pdf


# Grid search

In [11]:
jobs = execute(get_config('grid_configs/multi_fashion/uniform.yaml'), seeds)

In [12]:
jobs = execute(get_config('grid_configs/multi_fashion/single_task_1.yaml'), seeds)
jobs = execute(get_config('grid_configs/multi_fashion/single_task_2.yaml'), seeds)

In [13]:
jobs = execute(get_config('grid_configs/multi_fashion_mnist/uniform.yaml'), seeds)

In [14]:
jobs = execute(get_config('grid_configs/multi_fashion_mnist/single_task_1.yaml'), seeds)
jobs = execute(get_config('grid_configs/multi_fashion_mnist/single_task_2.yaml'), seeds)

In [15]:
jobs = execute(get_config('grid_configs/multi_mnist/uniform.yaml'), seeds)

In [16]:
jobs = execute(get_config('grid_configs/multi_mnist/single_task_1.yaml'), seeds)
jobs = execute(get_config('grid_configs/multi_mnist/single_task_2.yaml'), seeds)

## Results
#### Loss

In [11]:
results = load_data(dirname='results_grid', custom_metric=True)
plot_row(results, prefix='grid-mcr')
generate_table(results, name='tab-grid')

loaded data for multi_mnist
loaded data for multi_fashion
loaded data for multi_fashion_mnist
success. See grid-mcr.pdf
